In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
import tempfile

import numpy as np

from ska_pydada import AsciiHeader, DadaFile

## Create a DADA file

The cells below show how to create a DADA file with some random data.

The steps to create a DADA file are:
* create a header
* set the data
* serialise/dump to an output file

### Create ASCII Header

A header for the DADA file format is a simple key value structure that has at least 4096 bytes. The size
of the ASCII header is defined by the `HDR_SIZE` key and in the PyDADA library one can override this value
at construction time or via setting the property later.

```python
header = AsciiHeader(header_size=16384)
```

or

```python
header = AsciiHeader()
header.header_size = 16384
```

in the either example above the serialised header will have exactly 16384 bytes.

A header can be created from a string or a byte array (this is what is used when the `DadaFile` loads a DADA file).  The following
will use the `header_txt` variable to load the header.

In [3]:
header_txt = """HDR_SIZE            16384
HDR_VERSION         1.0
NCHAN               432
NBIT                32
NDIM                2
NPOL                2
RESOLUTION          1327104
UTC_START           2017-08-01-15:53:29
"""

In [4]:
header = AsciiHeader.from_str(header_txt)

In [5]:
assert header.header_size == 16384
assert int(header["NCHAN"]) == 432
assert int(header["NBIT"]) == 32
assert int(header["NDIM"]) == 2
assert int(header["NPOL"]) == 2
assert header.resolution == 1327104
assert header["UTC_START"] == "2017-08-01-15:53:29"

There are also utility methods on the `AsciiHeader` to get a header record as an `int` or `float`.

In [6]:
nbit = header.get_int("NBIT")
assert nbit == 32

Values can be added to the header either using a Python `dict` item insert or using the `set_value`

In [7]:
header["SOURCE"] = "J1644-4559_R"
assert header.get_value("SOURCE") == "J1644-4559_R"

# or

header.set_value("DESCRIPTION", "Some fancy description")
assert header["DESCRIPTION"] == "Some fancy description"

### Generate some data

For this notebook, the data will be random complex data with 768 time bins, 432 frequency channels, and 2 polarisations.

In [8]:
data = np.random.rand(768, 432, 2 * 2).astype(np.float32).view(np.complex64)
data.shape

(768, 432, 2)

In [9]:
data

array([[[4.15014207e-01+0.92422456j, 3.51797193e-01+0.22481613j],
        [7.78446138e-01+0.65127456j, 6.74277663e-01+0.60418534j],
        [7.48650432e-01+0.43587106j, 3.81811871e-04+0.75635874j],
        ...,
        [6.32126108e-02+0.34834364j, 2.14699641e-01+0.94942826j],
        [5.32752201e-02+0.9573674j , 9.14725125e-01+0.9137089j ],
        [2.40919471e-01+0.4547225j , 4.49067682e-01+0.18571945j]],

       [[2.32930064e-01+0.81249505j, 1.49466917e-01+0.6741856j ],
        [6.47610307e-01+0.05604906j, 8.94148573e-02+0.75386304j],
        [4.15247977e-01+0.7848931j , 1.93183616e-01+0.08598231j],
        ...,
        [3.21917802e-01+0.5064022j , 6.50760353e-01+0.2246425j ],
        [9.86626863e-01+0.80133253j, 8.01851153e-01+0.67760986j],
        [7.50192523e-01+0.6449764j , 8.46485198e-01+0.17329976j]],

       [[1.77767292e-01+0.93690455j, 4.37682956e-01+0.11093226j],
        [1.79274842e-01+0.29705492j, 1.35664403e-01+0.19951363j],
        [8.98604751e-01+0.29314902j, 7.4985063

An instance of a `DadaFile` can be created using the constructor that takes an optional `AsciiHeader` and an optional by array of data.

However, the following show how to create a `DadaFile` and then set the data afterwards.

In [10]:
dada_file = DadaFile(header=header)
dada_file.set_data(data)

Once an instance of a `DadaFile` has been created, it can be saved as a file. This can be then be read back later

In [11]:
assert dada_file.data_size == len(data.tobytes())
dada_file.data_size

5308416

In [12]:
tmpdir = tempfile.gettempdir()
outfile = pathlib.Path(tmpdir) / "example_dada_file.dada"

dada_file.dump(outfile)

In [13]:
%ls -lh $outfile

-rw-rw-r-- 1 wgauvin wgauvin 5.1M Mar 21 15:53 /tmp/example_dada_file.dada


In [14]:
!head -c4096 $outfile

HDR_SIZE            16384
HDR_VERSION         1.0
NCHAN               432
NBIT                32
NDIM                2
NPOL                2
RESOLUTION          1327104
UTC_START           2017-08-01-15:53:29
SOURCE              J1644-4559_R
DESCRIPTION         Some fancy description
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

## Loading and reading DADA files

While the above shows how to create DADA files that is normally used for testing and is not the main focus of the PyDADA library or `DadaFile` itself. The power of the `DadaFile` is that it can read DADA files that conform to the DADA spec of having a header that has `HDR_SIZE` value of at least 4096 bytes and that the binary data is afterwards. As it is a flexible file format the data may be packed in different ways and this API provides general data access methods to get the data.

The following cell will read the file generated above and print out the header.

In [15]:
dada_file2 = DadaFile.load_from_file(outfile)
print(dada_file2.header)

HDR_SIZE            16384
HDR_VERSION         1.0
NCHAN               432
NBIT                32
NDIM                2
NPOL                2
RESOLUTION          1327104
UTC_START           2017-08-01-15:53:29
SOURCE              J1644-4559_R
DESCRIPTION         Some fancy description



To get the data in time, frequency and polarisation structure, once can use the `as_time_freq_pol` help method. There is no need to know how the data is layed out provided the header records are correct.

In [16]:
tfp_data = dada_file2.as_time_freq_pol()
tfp_data

array([[[4.15014207e-01+0.92422456j, 3.51797193e-01+0.22481613j],
        [7.78446138e-01+0.65127456j, 6.74277663e-01+0.60418534j],
        [7.48650432e-01+0.43587106j, 3.81811871e-04+0.75635874j],
        ...,
        [6.32126108e-02+0.34834364j, 2.14699641e-01+0.94942826j],
        [5.32752201e-02+0.9573674j , 9.14725125e-01+0.9137089j ],
        [2.40919471e-01+0.4547225j , 4.49067682e-01+0.18571945j]],

       [[2.32930064e-01+0.81249505j, 1.49466917e-01+0.6741856j ],
        [6.47610307e-01+0.05604906j, 8.94148573e-02+0.75386304j],
        [4.15247977e-01+0.7848931j , 1.93183616e-01+0.08598231j],
        ...,
        [3.21917802e-01+0.5064022j , 6.50760353e-01+0.2246425j ],
        [9.86626863e-01+0.80133253j, 8.01851153e-01+0.67760986j],
        [7.50192523e-01+0.6449764j , 8.46485198e-01+0.17329976j]],

       [[1.77767292e-01+0.93690455j, 4.37682956e-01+0.11093226j],
        [1.79274842e-01+0.29705492j, 1.35664403e-01+0.19951363j],
        [8.98604751e-01+0.29314902j, 7.4985063

In [17]:
np.testing.assert_allclose(tfp_data, data)

The TFP data can also be retrieved by using the `data_c64` method one should include the shape.

In [18]:
tfp_data2 = dada_file2.data_c64(shape=(-1, 432, 2))
tfp_data2.shape

(768, 432, 2)

In [19]:
np.testing.assert_allclose(tfp_data2, data)

The raw data can be retrieved from the file by using:

In [20]:
raw_data = dada_file2.raw_data
len(raw_data), raw_data[:20]

(5308416, b'\xbe|\xd4>\xfb\x99l?\xc3\x1e\xb4>36f>?HG?')

### Large files

This notebook as uses small files but data recorded during a scan can result in large files.  Loading the whole file into memory is not efficient
and the `DadaFile` defaults to only loading data of around 4MB at a time, the amount of data loaded will be equal to a multiple of `RESOLUTION` value defined in the header (this does default to 1 byte if not set).

The below shows using the `load_next` method to get the next lot of data.

In [21]:
data = np.random.rand(768 * 2, 432, 2 * 2).astype(np.float32).view(np.complex64)
data.shape

dada_file.set_data(data)
dada_file.dump(outfile)

/tmp/example_dada_file.dada already exists, overwriting it


In [22]:
%ls -lh $outfile

-rw-rw-r-- 1 wgauvin wgauvin 11M Mar 21 15:53 /tmp/example_dada_file.dada


In [23]:
dada_file3 = DadaFile.load_from_file(outfile)
len(dada_file3.raw_data)

5308416

In [24]:
raw_data1 = dada_file3.raw_data
bytes_read = dada_file3.load_next()
assert bytes_read == 5308416
raw_data2 = dada_file3.raw_data

np.testing.assert_raises(AssertionError, np.testing.assert_array_equal, raw_data1, raw_data2)

bytes_read = dada_file3.load_next()
assert bytes_read == 0
raw_data3 = dada_file3.raw_data
np.testing.assert_array_equal(raw_data2, raw_data3)

In [25]:
raw_data1[:10], raw_data2[:10], raw_data3[:10]

(b'O\xbc\xdc=f\x86\x06?\xeb3', b'KMX?5\xd9)?\x87O', b'KMX?5\xd9)?\x87O')

From the above raw data we see that first call to `load_next` returns new data but the next call doesn't.

Methods like the `as_time_freq_pol` can still be used after a read but it is on the latest chuck of data.

In [26]:
tfp = dada_file3.as_time_freq_pol()
tfp

array([[[8.44929397e-01+0.66347057j, 5.71525991e-01+0.5648076j ],
        [7.64057279e-01+0.33549872j, 4.34140623e-01+0.15264596j],
        [2.18995377e-01+0.52870435j, 4.44219559e-01+0.25687957j],
        ...,
        [6.20718002e-01+0.12773935j, 8.54798257e-01+0.26467234j],
        [2.48293323e-03+0.45363408j, 1.09274164e-01+0.2750059j ],
        [7.08380878e-01+0.85032743j, 7.76992366e-02+0.5694364j ]],

       [[7.01006949e-01+0.0181182j , 6.54470563e-01+0.15279536j],
        [5.96601248e-01+0.20670807j, 5.37057638e-01+0.40137386j],
        [3.50261271e-01+0.9107441j , 5.60508192e-01+0.56139433j],
        ...,
        [1.35293379e-01+0.46010536j, 5.17713487e-01+0.44715196j],
        [5.62902391e-01+0.295607j  , 2.56077290e-01+0.21567811j],
        [2.98044086e-01+0.05080754j, 7.96310365e-01+0.2618667j ]],

       [[8.49327564e-01+0.33641246j, 7.84874022e-01+0.5078472j ],
        [4.74418253e-02+0.40870857j, 2.35575244e-01+0.18688285j],
        [5.74887753e-01+0.19846126j, 6.5572786

Note that the length of the raw dada is just over 5MB of data (it is `4 * RESOLUTION`). However, the file is around 11MB in size. More data can be loaded and processed by using the `load_next`

# Clean up

In [27]:
if outfile.exists():
    outfile.unlink()